# Create YOLO Dataset

In [ ]:
import os
import zipfile
from pylabel import importer
import shutil

In [ ]:
import os
from pylabel import importer

def import_dataset(path_to_annotations, path_to_images, dataset_name):
    dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name=dataset_name)
    return dataset

def split_and_export(dataset, output_path):
    dataset.export.ExportToYoloV5(output_path=output_path, yaml_file='dataset.yaml', copy_images=True)

# For train dataset
train_annotations = "COCOAnnotations/trainnew.json"
train_images = os.getcwd() + "/MergeDataset/"
train_dataset = import_dataset(train_annotations, train_images, "BCCD_coco")
split_and_export(train_dataset, 'YOLO_dataset_train/labels')

# For test dataset
test_annotations = "COCOAnnotations/testnew.json"
test_images = os.getcwd() + "/MergeDataset/"  # Assuming test images are in the same folder as train images
test_dataset = import_dataset(test_annotations, test_images, "BCCD_coco")
split_and_export(test_dataset, 'YOLO_dataset_test/labels')


In [ ]:
import os
import shutil
from pathlib import Path

# Define source and destination paths
test_source = 'YOLO_dataset_test/labels'
train_source = 'YOLO_dataset_train/labels'
test_dest = 'YOLO_dataset/labels/test'
train_dest = 'YOLO_dataset/labels/train'
source_file = "COCOAnnotations/dataset.yaml"
destination_dir = "YOLO_dataset"

# Create main directories
os.makedirs('YOLO_dataset/images/train', exist_ok=True)
os.makedirs('YOLO_dataset/images/test', exist_ok=True)

# Copy images from test and train directories
shutil.copytree('YOLO_dataset_test/images', 'YOLO_dataset/images/test', dirs_exist_ok=True)
shutil.copytree('YOLO_dataset_train/images', 'YOLO_dataset/images/train', dirs_exist_ok=True)

# Copy label directories
try:
    shutil.copytree(test_source, test_dest)
    shutil.copytree(train_source, train_dest)
    print("Directories copied successfully!")
except FileExistsError:
    print("Destination directories already exist. Remove or rename them and try again.")
except Exception as e:
    print(f"An error occurred: {e}")

# Copy dataset.yaml file
shutil.copy(source_file, destination_dir)

shutil.rmtree('YOLO_dataset_test')
shutil.rmtree('YOLO_dataset_train')

# Train Using Ultralytics Library

In [3]:
import os
from ultralytics import YOLO

ROOT_DIR = os.getcwd() + '/YOLO_dataset'


# Load a model
model = YOLO('yolov5nu.pt')

# Freeze
freeze = [f'model.{x}.' for x in range(20)]  # layers to freeze
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
    if any(x in k for x in freeze):
        v.requires_grad = False


# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=100,project='saved_models/YOLO/',device='cuda',batch=32,workers=10)  # train the model

New https://pypi.org/project/ultralytics/8.1.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.29 🚀 Python-3.9.19 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=c:\Users\rs659\Desktop\Object-Detection/YOLO_dataset\dataset.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=10, project=saved_models/YOLO/, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fal

train: Scanning C:\Users\rs659\Desktop\Object-Detection\YOLO_dataset\labels\train... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<00:00, 1673.60it/s]

train: WARNING ⚠️ C:\Users\rs659\Desktop\Object-Detection\YOLO_dataset\images\train\73066960192__D0573904-8DD2-497A-BC8F-58EADD430948.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\rs659\Desktop\Object-Detection\YOLO_dataset\images\train\73067139673__7775A454-0438-4B66-B846-0F6AC2ABE4B4.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\rs659\Desktop\Object-Detection\YOLO_dataset\images\train\IMG_4183.JPG: 3 duplicate labels removed


train: New cache created: C:\Users\rs659\Desktop\Object-Detection\YOLO_dataset\labels\train.cache


val: Scanning C:\Users\rs659\Desktop\Object-Detection\YOLO_dataset\labels\test... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 1303.67it/s]

...

    100/100      4.28G      0.354     0.3563      0.851         51        640: 100%|██████████| 8/8 [00:01<00:00,  7.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.13it/s]

                   all         60        206      0.669      0.616      0.632      0.505



100 epochs completed in 0.073 hours.
Optimizer stripped from saved_models\YOLO\train\weights\last.pt, 6.3MB
Optimizer stripped from saved_models\YOLO\train\weights\best.pt, 6.3MB

Validating saved_models\YOLO\train\weights\best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.9.19 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.01it/s]


                   all         60        206      0.693      0.582      0.643      0.511
                Drinks         60         75       0.61      0.493      0.584      0.485
              Utensils         60        112      0.526      0.384      0.424      0.336
                Laptop         60         19      0.943      0.869      0.922      0.712
Speed: 0.3ms preprocess, 0.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to saved_models\YOLO\train


# Train Using Ultralytics repo

In [2]:
!git clone https://github.com/ultralytics/yolov8

Cloning into 'yolov8'...
remote: Repository not found.
fatal: repository 'https://github.com/ultralytics/yolov8/' not found


In [3]:
import git

In [ ]:
# yolo detect train batch=16 epochs=500  data=YOLO_dataset/dataset.yaml model=YOLOv8m.pt  project=saved_models/YOLO/  name=YOLOv8m

In [ ]:
# python yolov5/train.py --batch 16 --epochs 100  --data YOLO_dataset/dataset.yaml  --hyp YOLO_hyp.yaml  --weights yolov5l.pt  --project  saved_models/YOLO/  --name YOLOv5l

# Training from list of models

In [ ]:
import os
from ultralytics import YOLO

# ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset'
ROOT_DIR = os.getcwd() + '\YOLO_dataset'



model_list = [
    # 'yolov5nu.pt',
    # 'yolov5s6u.pt',
    # 'yolov6-n.pt',
    # 'yolov6-n.pt',
    'yolov8n.pt',
    # 'yolov8s.pt',
    # 'yolov8m.pt',
    # 'yolov9c.pt'
]

for model_name in model_list:

    if os.path.exists(model_name.split(".")[0]):
        print("Folder exists")
        continue

    model = YOLO(model_name) 
    # Freeze
    try:
        freeze = [f'model.{x}.' for x in range(15)]  # layers to freeze
        for k, v in model.named_parameters():
            v.requires_grad = True  # train all layers
            if any(x in k for x in freeze):
                # print(f'freezing {k}')
                v.requires_grad = False
    except:
        model = YOLO(model_name) 


    # Use the model
    results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=500,project='saved_models/YOLO/',device='cuda',batch=8,workers=10,name=model_name.split('.')[0])  # train the model

## List Evaluation

In [ ]:
import os
from ultralytics import YOLO


model_list = [
    'yolov5nu.pt',
    'yolov5s6u.pt',
    # 'yolov6n.pt',
    # 'yolov6s.pt',
    'yolov8n.pt',
    'yolov8s.pt',
    'yolov8m.pt',
    'yolov9c.pt'
]


# Perform validation
for model_name in model_list:
    model = YOLO(f'saved_models/YOLO/{model_name.split(".")[0]}/weights/best.pt')
    results = model.val()

In [22]:
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

def annotate_image_with_predictions(model, image_path, output_path):
    """
    Annotate an image with object detection predictions using YOLOv5 model.

    Args:
        model_path (str): Path to the YOLOv5 model file.
        image_path (str): Path to the input image file.
        output_path (str): Path to save the annotated image.

    Returns:
        None
    """
    # Load the model
    # model = YOLO(model_path)

    # Read the image
    img = cv2.imread(image_path)

    # Perform object detection
    result = model.predict(img)[0]

    # Define color list for annotations
    color_list = [(0, 255, 0)] * len(result.boxes.data)  # Example: All annotations are green

    # Create annotator
    annotator = Annotator(img)

    # Add annotations for each detected object
    for label in result.boxes.data:
        annotator.box_label(
            label[0:4],
            f"{result.names[label[-1].item()]} {round(label[-2], 2)}",
            color_list[label[-1].item()]
        )

    # Save the annotated image
    cv2.imwrite(output_path, annotator.im)

# Example usage:
# model_path = "./path/to/model.pt"
# image_path = "./path/to/input/image.jpg"
image_path = 'MergeDataset/IMG_1854.JPG'
output_path = "otuptu.jpg"

annotate_image_with_predictions(model, image_path, output_path)
